In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from io import BytesIO
import openpyxl
from datetime import datetime

In [3]:
import Customer
import Codes
import ReadData
import Settings
import Subscription

In [4]:
writer = pd.ExcelWriter('VerdelingKlanten.xlsx', engine='openpyxl', mode='a')

In [5]:
path = 'CAOWijzer dataset 2017 tm 2021P06.xlsx'

In [6]:
app = ReadData.ReadData(path_file=path)

In [7]:
app.process_customers()

In [8]:
first_df = app.get_dataframe_from_customer_list()

In [9]:
first_df = first_df[first_df['Abonnement'].notna()]

In [10]:
first_df = first_df.sort_values(by='Laatst gefactureerde datum')

In [11]:
last_month = first_df['Laatst gefactureerde datum'].unique()[-1]

In [12]:
active_df = first_df[first_df['Laatst gefactureerde datum'] == last_month]

In [13]:
active_df.to_excel(writer, sheet_name='Data_actieve_klanten', startcol=4)

In [14]:
def get_total_values(df: pd.DataFrame) -> list:
    df_to_use = df.copy(deep=True)
    
    column_names = ['Slapende klanten', 'Light', 'Standard', 'Premium']
    
    values = [len(df_to_use[df_to_use['Gem. Gebruik'] == 0])]
    
    df_to_use = df_to_use[df_to_use['Gem. Gebruik'] != 0]
    
    values.extend([len(df_to_use[df_to_use['Abonnement'] == name]) for name in column_names[1:]])
    return [column_names, values]

In [15]:
def create_plot(column_names: list, values: list, title: str):
    image_name = BytesIO()
    plt.figure(figsize=(9, 6))
    plt.bar(column_names, values)
    plt.title(title)
    plt.xlabel('Verdeling vindt plaats obv huidige abonnement.\nActieve klanten zijn de klanten die in de maand juli 2021 zijn gefactureerd.')

    for n, v in zip(column_names, values):
        plt.text(n, v + 0.75, str(v))
    
    plt.savefig(image_name,  bbox_inches='tight', pad_inches=0.1, format='png')
    plt.close()
    
    image_name.seek(0)

    worksheet = writer.sheets['Data_actieve_klanten']
    image = openpyxl.drawing.image.Image(image_name)
    image.anchor = 'B2'
    worksheet.add_image(image)

    writer.save()

In [16]:
names, values = get_total_values(active_df)
create_plot(names, values, 'Verdeling van actieve klanten')

In [17]:
sleeping_df = active_df[active_df['Gem. Gebruik'] == 0]

In [18]:
sleeping_df

,Debiteur nr.,Klant,Abonnement,Totaal betaald,Gem. Prijs,Maanden boven bundel,Aantal maanden actief,Gem. Gebruik,Mediaan gebruik,Modus gebruik,Hoogste gebruik,Laatst gefactureerde datum,Subscription Object
293,12307,IBN Arbeidsintegratie (CAOWijzer),Light,425.0,22.37,0,19,0.0,0,0,0,2021-07-31,"[Subscription(price=0, free_clicks=0, extra_cl..."
303,12321,Robert Walters (CAOWijzer),Standard,800.0,44.44,0,18,0.0,0.0,0,0,2021-07-31,"[Subscription(price=0, free_clicks=0, extra_cl..."
375,12568,NL Services BV,Light,550.0,25.00,0,22,0.0,0.0,0,0,2021-07-31,"[Subscription(price=25.0, free_clicks=0, extra..."
380,12579,Career Control F B.V. (CAOWijzer),Light,525.0,25.00,0,21,0.0,0,0,0,2021-07-31,"[Subscription(price=25.0, free_clicks=0, extra..."
368,12124,Adams Multilingual Recruitment BV,Light,575.0,25.00,0,23,0.0,0,0,0,2021-07-31,"[Subscription(price=25.0, free_clicks=0, extra..."
364,12542,Unie-Pool Personeel BV,Standard,1200.0,48.00,0,25,0.0,0,0,0,2021-07-31,"[Subscription(price=0, free_clicks=0, extra_cl..."
333,11020,Experis Nederland B.V.,Premium,700.0,31.82,0,22,0.0,0.0,0,0,2021-07-31,"[Subscription(price=0, free_clicks=0, extra_cl..."
251,1180162,ABU (CAOWijzer),Light,150.0,5.56,0,27,0.0,0,0,0,2021-07-31,"[Subscription(price=0, free_clicks=0, extra_cl..."
252,12289,AllinFlex (CAOWijzer),Light,475.0,21.59,0,22,0.0,0.0,0,0,2021-07-31,"[Subscription(price=0, free_clicks=0, extra_cl..."
279,12223,Nexum Flex Forces BV OUD NIET GEBRUIKEN,Standard,950.0,43.18,0,22,0.0,0.0,0,0,2021-07-31,"[Subscription(price=0, free_clicks=0, extra_cl..."


In [19]:
amount_light = len(sleeping_df[sleeping_df['Abonnement'] == 'Light'])
amount_standard = len(sleeping_df[sleeping_df['Abonnement'] == 'Standard'])
amount_premium = len(sleeping_df[sleeping_df['Abonnement'] == 'Premium'])

In [20]:
title = 'Verdeling van de slapende klanten'
column_names = ['Light', 'Standard', 'Premium']
values = [amount_light, amount_standard, amount_premium]

In [21]:
def create_plot(column_names: list, values: list, title: str):
    image_name = BytesIO()
    plt.figure(figsize=(9,6))
    plt.bar(column_names, values)
    plt.title(title)
    for n, v in zip(column_names, values):
        plt.text(n, v + 0.05, str(v))

    plt.xlabel('\nDit zijn de slapende klanten die gefactureerd zijn in juli 2021')    
    
    plt.savefig(image_name,  bbox_inches='tight', pad_inches=0.1, format='png')
    plt.close()
    
    image_name.seek(0)

    worksheet = writer.sheets['Data_actieve_klanten']
    image = openpyxl.drawing.image.Image(image_name)
    image.anchor = 'B2'
    worksheet.add_image(image)

    writer.save()

In [22]:
create_plot(column_names=column_names, values=values, title=title)

In [23]:
writer.close()